In [46]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import pandas_ta as ta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor

root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()

# feature engineering and scaling
def prep(dataset):
    scaler = StandardScaler()

    macro_path1 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\IRP DATASET  - Copy of US_macroeconomics.csv"
    macro_path2 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\^VIX.csv"
    macro_path3 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\CORESTICKM159SFRBATL.csv"
    macro_path4 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\FFR.csv"
    macro_path5 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GDP.csv"
    macro_path6 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VIX9D_History.csv"
    macro_path7 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GVZ_History.csv"
    macro_path8 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\OVX_History.csv"
    macro_path9 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VVIX_History.csv"
    macro_data1 = pd.read_csv(macro_path1)
    macro_data2 = pd.read_csv(macro_path2)
    macro_data3 = pd.read_csv(macro_path3)
    macro_data4 = pd.read_csv(macro_path4)
    macro_data5 = pd.read_csv(macro_path5)
    macro_data6 = pd.read_csv(macro_path6)
    macro_data7 = pd.read_csv(macro_path7)
    macro_data8 = pd.read_csv(macro_path8)
    macro_data9 = pd.read_csv(macro_path9)

    dataset['Date'] = pd.to_datetime(dataset['Date'], dayfirst=True)
    dataset.set_index('Date', inplace=True)
    dataset['Target'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)

    macro_data1['date'] = pd.to_datetime(macro_data1['date'], dayfirst=True)
    macro_data1.set_index('date', inplace=True)

    macro_data2['Date'] = pd.to_datetime(macro_data2['Date'], dayfirst=True)
    macro_data2.set_index('Date', inplace=True)

    macro_data3['Date'] = pd.to_datetime(macro_data3['Date'], dayfirst=True)
    macro_data3.set_index('Date', inplace=True)

    macro_data4['Date'] = pd.to_datetime(macro_data4['Date'], dayfirst=True)
    macro_data4.set_index('Date', inplace=True)

    macro_data5['Date'] = pd.to_datetime(macro_data5['Date'], dayfirst=True)
    macro_data5.set_index('Date', inplace=True)

    macro_data7['Date'] = pd.to_datetime(macro_data7['Date'], dayfirst=False)
    macro_data7.set_index('Date', inplace=True)

    macro_data8['Date'] = pd.to_datetime(macro_data8['Date'], dayfirst=False)
    macro_data8.set_index('Date', inplace=True)

    macro_data9['Date'] = pd.to_datetime(macro_data9['Date'])
    macro_data9.set_index('Date', inplace=True)

    merge_data1 = dataset.join(macro_data1, how='left').fillna(method='ffill')
    merge_data2 = merge_data1.join(macro_data2, how='left').fillna(method='ffill')
    merge_data3 = merge_data2.join(macro_data3, how='left').fillna(method='ffill')
    merge_data4 = merge_data3.join(macro_data4, how='left').fillna(method='ffill')
    merge_data5 = merge_data4.join(macro_data5, how='left').fillna(method='ffill')
    merge_data7 = merge_data5.join(macro_data7, how='left').fillna(method='ffill')
    merge_data8 = merge_data7.join(macro_data8, how='left').fillna(method='ffill')
    merge_final = merge_data8.join(macro_data9, how='left').fillna(method='ffill')

    merge_final['RSI (14D)'] = ta.rsi(merge_final['Close'], length=14)
    merge_final['20 Day CCI'] = ta.cci(high=merge_final['High'], low=merge_final['Low'], 
                                    close=merge_final['Close'], length=20)
    merge_final['Williams %R'] = ta.willr(high=merge_final['High'], low=merge_final['Low'], 
                                        close=merge_final['Close'], length=14)
    merge_final['EMA (5D)'] = merge_final['Close'].ewm(span=5, adjust=False).mean()
    merge_final['MA50'] = merge_final['Close'].rolling(window=50).mean()

    final_dataset = merge_final.dropna()

    time_shifted = final_dataset#.resample('M').asfreq().dropna()
    
    features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
                'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
                'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'FFR']
    
    time_shifted[features] = time_shifted[features].astype(float)
    time_shifted[features] = scaler.fit_transform(time_shifted[features])

    return time_shifted, scaler

# get dataset
file_path = filedialog.askopenfilename(parent=root, title="Select A File")
ticker = pd.read_csv(file_path)
ticker, scaler = prep(ticker)

ticker

C:\Users\samto\AppData\Local\Temp\ipykernel_24448\2806024467.py:70: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merge_data1 = dataset.join(macro_data1, how='left').fillna(method='ffill')
C:\Users\samto\AppData\Local\Temp\ipykernel_24448\2806024467.py:71: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merge_data2 = merge_data1.join(macro_data2, how='left').fillna(method='ffill')
C:\Users\samto\AppData\Local\Temp\ipykernel_24448\2806024467.py:72: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merge_data3 = merge_data2.join(macro_data3, how='left').fillna(method='ffill')
C:\Users\samto\AppData\Local\Temp\ipykernel_24448\2806024467.py:73: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a f

,Open,High,Low,Close,Adj Close,Volume,Target,CPI,Mortgage_rate,Unemp_rate,...,FFR,GDP,GVZ,OVX,VVIX,RSI (14D),20 Day CCI,Williams %R,EMA (5D),MA50
Date,,,,,,,,,,,,,,,,,,,,,
2009-09-18,-1.3093,-1.3101,-1.3058,-1.3082,-1.3082,1.7228,0,-1.4824,1.4564,1.6576,...,-0.5857,14448.8820,1.1173,0.2701,-1.4931,1.4074,1.1448,0.8895,-1.3149,-1.3656
2009-09-21,-1.3089,-1.3137,-1.3116,-1.3113,-1.3113,0.6899,1,-1.4824,1.4564,1.6576,...,-0.5919,14448.8820,1.2433,0.2543,-1.4555,1.1813,0.7634,0.7470,-1.3136,-1.3624
2009-09-22,-1.3095,-1.3082,-1.3040,-1.3053,-1.3053,1.3468,0,-1.4824,1.4564,1.6576,...,-0.5919,14448.8820,1.5182,0.1945,-1.6243,1.3705,0.8375,1.0211,-1.3108,-1.3595
2009-09-23,-1.3041,-1.3028,-1.3091,-1.3145,-1.3145,1.6437,0,-1.4824,1.4564,1.6576,...,-0.5919,14448.8820,1.4028,0.2443,-1.6293,0.7244,0.5992,0.4317,-1.3120,-1.3568
2009-09-24,-1.3127,-1.3145,-1.3215,-1.3231,-1.3231,1.6163,0,-1.4824,1.4564,1.6576,...,-0.5981,14448.8820,0.9872,0.4231,-1.4691,0.1950,0.1579,-0.0701,-1.3157,-1.3548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-23,2.5118,2.5037,2.5260,2.5028,2.5028,-0.4703,0,1.7557,1.3415,-1.0050,...,2.6332,28629.1530,-0.3897,-0.6052,-0.3766,-0.0559,-0.1565,-0.9968,2.5126,2.4566
2024-07-24,2.4613,2.4384,2.4142,2.3936,2.3936,-0.0069,0,1.7557,1.3415,-1.0050,...,2.6332,28629.1530,-0.2764,-0.5988,0.7084,-1.2518,-1.5002,-2.0064,2.4748,2.4602
2024-07-25,2.3958,2.4245,2.3894,2.3699,2.3699,0.6658,1,1.7557,1.3415,-1.0050,...,2.6332,28629.1530,-0.3687,-0.6199,0.6012,-1.4515,-1.6974,-2.0030,2.4417,2.4632


In [3]:
vif_ticker = pd.DataFrame()
features = ticker[['Close', 'Volume', 'CPI', 
                'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
                'RSI (14D)', '20 Day CCI', 'Williams %R', 'MA50', 'FFR']]
vif_ticker["feature"] = features.columns

vif_ticker["VIF"] = [variance_inflation_factor(features.values, i)
                          for i in range(len(features.columns))]

In [47]:
# create LSTM model class
class LSTM_Model(nn.Module):
    def __init__(self, input_layer, hidden_layer, output_layer, num_layers=4, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.hidden_layer = hidden_layer
        self.num_layers = num_layers
        
        # LSTM layers with dropout
        self.lstm = nn.LSTM(input_layer, hidden_layer, num_layers, batch_first=True, dropout=dropout)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Linear layer
        self.linear_layer = nn.Linear(hidden_layer, output_layer)

    def forward(self, input_tensor):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        c0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        
        # Pass through LSTM layers
        out, _ = self.lstm(input_tensor, (h0, c0))
        
        # Apply dropout
        out = self.dropout(out[:, -1, :])
        
        # Pass through linear layer
        out = self.linear_layer(out)
        return out

# create sequences for input data and corresponding labels
def create_sequence(input_data, sequence_length):
    sequences = []
    for i in range(0, len(input_data) - sequence_length):
        sequence = input_data[i : i + sequence_length, :-1]
        label = input_data[i + sequence_length, -1]
        sequences.append((sequence, label))
    return sequences

# train the model with data provided
def trainer(model, train_data, loss_func, opt, epochs, fold, losses):
    model.train()
    for epoch in range(epochs):
        for sequence, labels in train_data:
            opt.zero_grad()
            sequence = sequence.clone().detach().float()
            labels = labels.clone().detach().float().view(-1, 1)
            
            y = model(sequence)
            loss = loss_func(y, labels)
            loss.backward()
            opt.step()

        if epoch % 10 == 1 or epoch == 99:
            print(f'Epoch {epoch} loss: {loss.item():.4f}')

            new_row = pd.DataFrame({'Fold': [fold+1], 'Epoch': [epoch], 'Loss': [loss.item()]})
            losses = pd.concat([losses, new_row], ignore_index=True)
            
    return losses

def predictor(model, test_data):
    model.eval()
    predictions = []
    with torch.no_grad():
        for sequence, _ in test_data:
            sequence = sequence.clone().detach().float()
            y = model(sequence)
            batch_predictions = torch.sigmoid(y)

            # Convert predictions to tensor if they are scalar or float
            if isinstance(batch_predictions, float) or batch_predictions.ndimension() == 0:
                batch_predictions = torch.tensor([batch_predictions])

            batch_predictions = torch.round(batch_predictions)

            predictions.extend(batch_predictions.squeeze().tolist())
    
    return predictions

# create sequences
sequence_length = 10
sequences = create_sequence(ticker[['Close', 'Volume', 'CPI', 'Mortgage_rate', 'Unemp_rate', 
                                    'disposable_income','GVZ', 'OVX', 'VVIX', 'RSI (14D)', 
                                    '20 Day CCI', 'Williams %R', 'MA50', 'FFR', 'Target']].values, sequence_length)

# cross-validation
tscv = TimeSeriesSplit(n_splits=10)
accuracies = []
precisions = []
recalls = []
f1s = []
kappas = []
losses = pd.DataFrame(columns=['Fold', 'Epoch', 'Loss'])

for fold, (train_index, test_index) in enumerate(tscv.split(sequences)):
    print(f'Fold {fold+1}')
    
    train_sequences = [sequences[i] for i in train_index]
    test_sequences = [sequences[i] for i in test_index]
    
    train_data = torch.utils.data.DataLoader(train_sequences, shuffle=True, batch_size=32)
    test_data = torch.utils.data.DataLoader(test_sequences, shuffle=False, batch_size=32)
    
    # initialize model
    model = LSTM_Model(input_layer=14, hidden_layer=50, output_layer=1)
    loss_func = nn.BCEWithLogitsLoss()
    opt = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    # train model
    epochs = 100
    losses = trainer(model, train_data, loss_func, opt, epochs, fold, losses)
    
    # make predictions
    test_labels = [label for _, label in test_sequences]
    predictions = predictor(model, test_data)
    
    # calculate statistics
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, zero_division=0)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    kappa = cohen_kappa_score(test_labels, predictions)
    
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    kappas.append(kappa)
    
    print(f'\nAccuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1: {f1:.4f}')
    print(f'Kappa: {kappa:.4f}\n')

# average scores across all folds
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1s)
avg_kappa = np.mean(kappas)

print(f'\nCross-Validation Results:')
print(f'Average Accuracy: {avg_accuracy:.4f}')
print(f'Average Precision: {avg_precision:.4f}')
print(f'Average Recall: {avg_recall:.4f}')
print(f'Average F1: {avg_f1:.4f}')
print(f'Average Kappa: {avg_kappa:.4f}')


Fold 1
Epoch 1 loss: 0.6104


C:\Users\samto\AppData\Local\Temp\ipykernel_24448\3942879950.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  losses = pd.concat([losses, new_row], ignore_index=True)


Epoch 11 loss: 0.6128
Epoch 21 loss: 0.6502
Epoch 31 loss: 0.5963
Epoch 41 loss: 0.8048
Epoch 51 loss: 0.5672
Epoch 61 loss: 0.5264
Epoch 71 loss: 0.6883
Epoch 81 loss: 0.6441
Epoch 91 loss: 0.6048
Epoch 99 loss: 0.5355

Accuracy: 0.5516
Precision: 0.5917
Recall: 0.5464
F1: 0.5682
Kappa: 0.1035

Fold 2
Epoch 1 loss: 0.6434
Epoch 11 loss: 0.5994
Epoch 21 loss: 0.6793
Epoch 31 loss: 0.5463
Epoch 41 loss: 0.6511
Epoch 51 loss: 0.4734
Epoch 61 loss: 0.7236
Epoch 71 loss: 0.6281
Epoch 81 loss: 0.6529
Epoch 91 loss: 0.5020
Epoch 99 loss: 0.5135

Accuracy: 0.5310
Precision: 0.5536
Recall: 0.8421
F1: 0.6681
Kappa: -0.0255

Fold 3
Epoch 1 loss: 0.7450
Epoch 11 loss: 0.6438
Epoch 21 loss: 0.6439
Epoch 31 loss: 0.6810
Epoch 41 loss: 0.5997
Epoch 51 loss: 0.6107
Epoch 61 loss: 0.5938
Epoch 71 loss: 0.6717
Epoch 81 loss: 0.6251
Epoch 91 loss: 0.6834
Epoch 99 loss: 0.4348

Accuracy: 0.5044
Precision: 0.5827
Recall: 0.3915
F1: 0.4684
Kappa: 0.0366

Fold 4
Epoch 1 loss: 0.7129
Epoch 11 loss: 0.6960
Ep

In [48]:
fall = (ticker.Target == 0).sum()
rise = (ticker.Target == 1).sum()

rise_per = (rise / (rise + fall)) * 100
fall_per = (fall / (rise + fall)) * 100

print(f'Number of days rising: {rise}')
print(f'Number of days falling: {fall}')

print(f'Rise % is: {rise_per:.2f}%')
print(f'Fall % is: {fall_per:.2f}%')

Number of days rising: 2036
Number of days falling: 1703
Rise % is: 54.45%
Fall % is: 45.55%


In [49]:
results = pd.DataFrame(columns=['Dataset', 'Average Accuracy', 'Average Precision', 
                                'Average Recall', 'Average F1', 'Average Kappa', 'Number of Rises', 
                                'Number of Falls', 'Rise %', 'Fall %'])

datatset_name = (file_path.split('/')[-1].split('.')[0])

metrics = {'Dataset': datatset_name ,'Average Accuracy': avg_accuracy, 
           'Average Precision': avg_precision, 'Average Recall': avg_recall, 
           'Average F1': avg_f1, 'Average Kappa': avg_kappa, 'Number of Rises': rise, 'Number of Falls': fall, 
           'Rise %': rise_per, 'Fall %': fall_per}

export_results = results._append(metrics, ignore_index=True)

export_results = pd.concat([export_results, losses], ignore_index=True)

export_results.set_index('Dataset', inplace=True)

export_results.to_csv(r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\results\\" + datatset_name + '_DAILY_LOSS.csv')

export_results

C:\Users\samto\AppData\Local\Temp\ipykernel_24448\4160461266.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  export_results = results._append(metrics, ignore_index=True)


,Average Accuracy,Average Precision,Average Recall,Average F1,Average Kappa,Number of Rises,Number of Falls,Rise %,Fall %,Fold,Epoch,Loss
Dataset,,,,,,,,,,,,
^SPX,0.5109,0.4904,0.6193,0.5320,0.0076,2036,1703,54.4531,45.5469,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.6104
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11,0.6128
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21,0.6502
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,31,0.5963
...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,61,0.6706
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,71,0.6946
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,81,0.6848
